In [1]:
import pandas as pd
import numpy as np
import warnings
import re
import math
# warnings.filterwarnings("ignore")
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000

In [2]:
chem_list = [
    u'C',
    u'Mn',
    u'Si',
    u'P',
    u'S',
    u'Cr',
    u'Ni',
    u'Cu',
    u'Al',
    u'V',
    u'Ti',
    u'Nb',
    u'Mo',
    u'N',
    u'B'
]

In [3]:
def ch2f(x):
    if (type(x)==float) | (type(x)==int):
        x = str(x)
    if u'г' in x:
        print(x)
        x = x.replace(u'г.п.', '')
        x = x.replace(u'гп', '')
        x = str(x)
    if u'0.012. 0.014. 0.013' in str(x):
        x = x.replace(u'0.012. 0.014. 0.013', u'0.013')
    if x[-4:-2]=='e-':
        return float(x)
    elif '-' in str(x): 
#         print(x)
        x = x.replace(' ', '')
        x = str(x) + '0'
        tmp_map = list(map(float, str(x).replace(',', '.').split('-')))
        if (abs(tmp_map[-1]-tmp_map[-2])>0.05) | (tmp_map[-1]==0):
            return np.mean(tmp_map[:-1])
        else:
            return np.mean(tmp_map)
    else:
        return float(str(x).replace(',', '.'))


In [4]:
# подгрузка всех файлов испытаний
import os
dr = 'DATA/source/exp'
file_names = os.listdir(dr)

In [5]:
file_names.remove('prepared')

In [6]:
file_names

['1квартал2017.xls',
 '2012 1 квартал (январь, февраль, март)испр.xls',
 '2012 2 квартал (апрель, май, июнь)испр..xls',
 '2012 3 квартал (июль, август, сентябрь)испр..xls',
 '2012 4 квартал (октябрь, ноябрь, декабрь)испр.xls',
 '2013 1 квартал (январь, февраль, март)испр.xls',
 '2013 2 квартал (апрель, май, июнь)испр..xls',
 '2013 3 квартал (июль, август, сентябрь)испр..xls',
 '2013 4 квартал (октябрь, ноябрь, декабрь)испр.xls',
 '2014 1 квартал (январь, февраль, март)испр.xls',
 '2014 2 квартал (апрель, май, июнь)испр.xls',
 '2014 3 квартал (июль, август, сентябрь)испр.xls',
 '2014 4 квартал (октябрь, ноябрь, декабрь)испр.xls',
 '2015 1 квартал (январь, февраль, март)испр.xls',
 '2015 2 квартал (апрель, май, июнь)испр.xls',
 '2015 3 квартал ( июль, август, сентябрь)испр.xls',
 '2015 4 квартал (октябрь, ноябрь, декабрь)испр.xls',
 '2016 1 квартал (январь,февраль,март)испр.xls',
 '2016 2 квартал (апрель, май, июнь)испр.xls',
 '2016 3 квартал (июль, август, сентябрь)испр.xls',
 '2016 4 к

In [7]:
# файлы в словарь доступ по имени файла ({имя файла: словарь датафреймов файла} )
exp_files = {}
for name in file_names:
    exp_files[name] = pd.read_excel(dr+'\\'+name, None)
#     print(name)

In [8]:
# номера плавок приводим к единому регистру (нижнему)
tmp = {}
for name in exp_files:
    for l in exp_files[name]:
        if '№ плавки' in exp_files[name][l]:
            exp_files[name][l]['№ плавки'] = exp_files[name][l]['№ плавки'].astype(str).str.lower()
        if '№ партии' in exp_files[name][l]:
            exp_files[name][l]['№ партии'] = exp_files[name][l]['№ партии'].astype(str).str.lower()
    tmp[name] = exp_files[name]
exp_files = tmp

In [9]:
# создаем единую таблицу с химией
chemistry = []
chemistry = exp_files['1квартал2017.xls'][u'Хим.анализ плавок (заготовка)']

# TODO : убрать этот список эта вкладка всегда первая!!!!!!!!!!!
"""TODO : убрать этот список эта вкладка всегда первая!!!!!!!!!!!"""
# TODO : убрать этот список эта вкладка всегда первая!!!!!!!!!!!
# TODO : убрать этот список эта вкладка всегда первая!!!!!!!!!!!

chemistry_list_names = [
    'Хим.анализ (заготовка)',
    'Хим.анализ плавок (заготовка)',
    'Хим.анализ плавок(заготовка)',
    'Хим.анализ плавок (Заготовка)',
    'Хим.анализ плавок (заготовки)',
    'Хим.анализ-заготовка',
    'Хим.анализ плавок'
]

s = 0
for name, exp_file in zip(exp_files, exp_files.values()):
    for list_chem in chemistry_list_names:
        if list_chem in exp_file:
            chemistry = pd.merge(chemistry, exp_file[list_chem], how = 'outer')
            s += exp_file[list_chem].shape[0]
            break

chemistry[u'№ плавки'] = chemistry[u'№ плавки'].astype(str)
chemistry.to_excel('chemistry.xlsx')

print(s)
print(chemistry.shape)

17182
(12998, 163)


In [10]:
# chemistry = chemistry[chemistry['Поставщик']=='ЭСПК']
print(chemistry.shape)

(12998, 163)


In [11]:
for el in chem_list:
    chemistry[el]=chemistry[el].apply(ch2f)

0.18 г.п.
0.22 гп
0.21 г.п.


In [12]:
def get_more_chem(x):
    for el in x:
        for i in x[el]:
            if i!=None:
                x[el] = i
    return x
                
chemistry = chemistry.groupby('№ плавки').apply(get_more_chem)

In [13]:
chemistry.drop_duplicates(inplace=True)
chemistry.shape

(10746, 163)

In [14]:
# chemistry[chemistry[['№ плавки']].duplicated(keep = False)].sort_values(['№ плавки'])

In [14]:
# подгружаем режимы
mode = pd.read_excel('all_modes_clean.xlsx')
mode.shape
mode[u'№ плавки'] = mode[u'№ плавки'].astype(str)
mode.shape

(29255, 33)

In [15]:
# подгружаем режимы valid
mode_valid = pd.read_excel('valid_modes_clean.xlsx')
mode_valid[u'№ плавки'] = mode_valid[u'№ плавки'].astype(str)
mode_valid.shape

(1722, 22)

In [17]:
# TODO:  очищаем дубликаты по номеру партии (скорее всего какие-то косяки есть)
# mode[mode[['№ плавки', '№ партии']].duplicated(keep = False)].sort_values(['№ партии'])

In [16]:
merge_table = pd.merge(mode, chemistry, how ='left', on=['№ плавки'])
print(merge_table.shape)
merge_table_valid = pd.merge(mode_valid, chemistry, how ='left', on=['№ плавки'])
print(merge_table_valid.shape)

(29255, 195)
(1722, 184)


In [17]:
# Среднюю химию подгружаем 
# mean_chem_steel = pd.read_excel('mean_chem_steel.xlsx')

In [18]:
# merge_table.shape
# , mean_chem_steel.shape
1722 - 1698

24

In [21]:
# ls = merge_table.columns[:-15]
# merge_without_chem = merge_table[merge_table['C'].isnull()]
# print(merge_without_chem.shape)
# merge_without_chem = merge_without_chem[ls]
# merge_without_chem.shape

In [19]:
merge_table = merge_table[merge_table['C'].notnull()]
print(merge_table.shape)
merge_table_valid = merge_table_valid[merge_table_valid['C'].notnull()]
print(merge_table_valid.shape)

(28352, 195)
(1698, 184)


In [23]:
# merge_mean_chem = pd.merge(merge_without_chem, mean_chem_steel, how='left', on='марка стали')

In [24]:
# merge_mean_chem.shape

In [25]:
# merge_table = pd.merge(merge_table, merge_mean_chem, how ='outer')
# merge_table.shape

In [20]:
copy_merge_table = merge_table.copy()
copy_merge_table_valid = merge_table_valid.copy()
copy_merge_table.shape, copy_merge_table_valid.shape

((28352, 195), (1698, 184))

In [21]:
merge_table = copy_merge_table.copy()
merge_table_valid = copy_merge_table_valid.copy()
merge_table.shape, merge_table_valid.shape

((28352, 195), (1698, 184))

In [22]:
def clean_by_chem(df, mark, el, right_b, left_b=None):
    tmp = df.shape[0]
    if left_b==None:
        df = df[((df['марка стали']==mark) & ((df[el]<=right_b)|(pd.isnull(df[el]))))| (df['марка стали']!=mark)]
    else:
        df = df[((df['марка стали']==mark) & ((df[el]<=right_b)|(pd.isnull(df[el]))) & ((df[el]>=left_b)|(pd.isnull(df[el])))) | (df['марка стали']!=mark)]
#     print(df.shape)
    if (df.shape[0]-tmp!=0):
        print('удалено строк:', np.abs(df.shape[0]-tmp), 'причина', el, mark)
    return df

In [23]:
def clean_all_mark_chem(merge_table):
    # чистим 09Г2C
    merge_table = clean_by_chem(merge_table, '09Г2С', 'C', 0.14, 0.03)
    merge_table = clean_by_chem(merge_table, '09Г2С', 'Mn', 1.8, 1)
    merge_table = clean_by_chem(merge_table, '09Г2С', 'Si', 0.75, 0.45)
    merge_table = clean_by_chem(merge_table, '09Г2С', 'S', 0.05)
    merge_table = clean_by_chem(merge_table, '09Г2С', 'P', 0.05)
    merge_table = clean_by_chem(merge_table, '09Г2С', 'Al', 0.06, 0.015)
    merge_table = clean_by_chem(merge_table, '09Г2С', 'Ti', 0.035)
    merge_table = clean_by_chem(merge_table, '09Г2С', 'Ni', 0.3)
    merge_table = clean_by_chem(merge_table, '09Г2С', 'Cr', 0.3)
    merge_table = clean_by_chem(merge_table, '09Г2С', 'Cu', 0.3)
    # чистим 13ХФА
    merge_table = clean_by_chem(merge_table, '13ХФА', 'C', 0.19, 0.1)
    merge_table = clean_by_chem(merge_table, '13ХФА', 'Mn', 0.7, 0.3)
    merge_table = clean_by_chem(merge_table, '13ХФА', 'Si', 0.4, 0.13)
    merge_table = clean_by_chem(merge_table, '13ХФА', 'Cr', 0.8, 0.4)
    merge_table = clean_by_chem(merge_table, '13ХФА', 'V', 0.08, 0.02)
    # чистим 30Г2
    merge_table = clean_by_chem(merge_table, '30Г2', 'C', 0.35, 0.25)
    merge_table = clean_by_chem(merge_table, '30Г2', 'Mn', 1.6, 0.7)
    merge_table = clean_by_chem(merge_table, '30Г2', 'Si', 0.5, 0.13)
    # чистим 18ХМФБ
    merge_table = clean_by_chem(merge_table, '18ХМФБ', 'C', 0.25, 0.12)
    merge_table = clean_by_chem(merge_table, '18ХМФБ', 'Mn', 0.7, 0.3)
    merge_table = clean_by_chem(merge_table, '18ХМФБ', 'Si', 0.5, 0.13)
    merge_table = clean_by_chem(merge_table, '18ХМФБ', 'Cr', 1.2, 0.45)
    merge_table = clean_by_chem(merge_table, '18ХМФБ', 'Mo', 0.4, 0.1)
    merge_table = clean_by_chem(merge_table, '18ХМФБ', 'V', 0.12, 0.02)
    merge_table = clean_by_chem(merge_table, '18ХМФБ', 'Nb', 0.08, 0.015)
    # чистим 37Г2Ф
    merge_table = clean_by_chem(merge_table, '37Г2Ф', 'C', 0.45, 0.35)
    merge_table = clean_by_chem(merge_table, '37Г2Ф', 'Mn', 1.8, 0.7)
    merge_table = clean_by_chem(merge_table, '37Г2Ф', 'Si', 0.5, 0.13)
    merge_table = clean_by_chem(merge_table, '37Г2Ф', 'V', 0.12, 0.02)
    return merge_table

In [30]:
# merge_table[merge_table['№ партии']==203010]

In [24]:
merge_table = clean_all_mark_chem(merge_table)
merge_table.shape

удалено строк: 21 причина C 09Г2С
удалено строк: 1 причина Mn 09Г2С
удалено строк: 1 причина Si 09Г2С
удалено строк: 27 причина Al 09Г2С
удалено строк: 4 причина Ti 09Г2С
удалено строк: 28 причина C 13ХФА
удалено строк: 1 причина Cr 13ХФА
удалено строк: 73 причина C 30Г2
удалено строк: 7 причина C 18ХМФБ
удалено строк: 38 причина Mn 18ХМФБ
удалено строк: 3 причина Cr 18ХМФБ
удалено строк: 362 причина C 37Г2Ф
удалено строк: 15 причина Mn 37Г2Ф
удалено строк: 5 причина Si 37Г2Ф
удалено строк: 13 причина V 37Г2Ф


(27753, 195)

In [25]:
merge_table_valid = clean_all_mark_chem(merge_table_valid)
merge_table_valid.shape

удалено строк: 6 причина C 37Г2Ф


(1692, 184)

In [26]:
merge_table_valid.shape

(1692, 184)

In [ ]:
# merge_table[merge_table['№ партии']==203010]

In [27]:
merge_table.to_excel('DATA/prepared/merge_table_mean_chem.xlsx')

In [28]:
merge_table_valid.to_excel('DATA/prepared/valid_table_mean_chem.xlsx')

In [44]:
merge_table.shape

(27753, 194)

In [43]:
merge_table_valid.to_excel('DATA/prepared/valid_table_mean_chem.xlsx')

In [45]:
merge_table.to_excel('DATA/prepared/merge_table_mean_chem.xlsx')